Парсер сайта napopravku.ru. Часть 2.

Собирает данные о клиниках:

- базовую информацию (адрес, телефон, время работы)
- тип клиники
- описание и преимущества
- среднюю оценку и количество отзывов
- информацию о врачах
- цены

Последние обновления скрипта: 2021-12-27

## Библиотеки и настройки ##

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/stom/'
proxy_list_path = 'C:/work/proxy_list.txt'

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]

In [ ]:
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси проще прописать вручную, ввод со строки постоянно глючит

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

#proxy_number = 1

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю данные ##

In [ ]:
# загружаю датафрейм с линками

works_links = pd.read_excel(file_path + 'napopravku_links_vladivostok.xlsx')

In [ ]:
works_links.head(2)

In [ ]:
works_links.shape

In [ ]:
# список с ссылками

links = works_links.clinic_page.to_list()

### Код парсера ###

In [ ]:
data = [] # список для данных


for url in tqdm(links):
    driver.get(url)
    sleep(3)

    # подтвежден ли профиль
    try:
        driver.find_element_by_class_name('n-tooltip.clinic-title__icon')
        verified = 1

    except:
        verified = 0


        # базовая информация - адрес, телефон, время работы

    # адрес
    
    try:
        address = (driver
                   .find_element_by_class_name('clinic-contacts')
                   .find_element_by_class_name('clinic-contacts__address').text
                  )
    except:
        address = 'none'


    # ближайшее метро
    try:
        metro_station = driver.find_element_by_class_name('location.clinic-contacts__metro').text

    except:
        metro_station = 'none'


    # телефон
    
    try:
        phone = driver.find_element_by_class_name('clinic-phone__value').text
        
    except:
        phone = 'none'
            

    # часы работы
    try:
        work_hours = driver.find_element_by_class_name('clinic-time').find_element_by_class_name('clinic-time__subtitle').text

    except:
        work_hours = 'none'



        # рейтинг и отзывы

    # рейтинг
    try:
        rating = driver.find_element_by_class_name('clinic-title__rating').find_element_by_class_name('rating__value').text

    except:
        rating = 'none'

    # количество отзывов
    try:
        review_number = driver.find_element_by_class_name('rating-info__review.text-md').text.replace(' отзывов', '')

    except:
        review_number = 0



        # профиль и тип клиники

    clinic_table_block = driver.find_elements_by_class_name('clinic-table__row') # вытаскиваю блок с инфой

    clinic_table = {}

    for item in clinic_table_block: #разбираю данные
        clinic_table[item.find_element_by_class_name('clinic-table__title').text] = (item
                                                                                .find_element_by_class_name('clinic-table__name')
                                                                                    .text)
        try:
            main_clinic = clinic_table['Головная клиника']

        except:
            main_clinic = 'none'

        try:
            clinic_type = clinic_table['Тип']

        except:
            clinic_type = 'none'

        try:
            age_profile = clinic_table['Принимает']

        except:
            age_profile = 'none'

        try:
            year_f = clinic_table['Год основания']

        except:
            year_f = 'none'



    # ссылка на страницу головной клиники
    try:
        main_clinic_page = (driver
                            .find_element_by_class_name('clinic-table__name.clinic-table__name--link')
                            .get_attribute('href'))
    except:
        main_clinic_page = 'none'



        # описание и преимущества

    # описание клиники
    try:
        driver.find_element_by_xpath('//span[@class="clinic-description__show link"]').click() # раскрываю описание
        description = (driver.find_element_by_class_name('clinic-description')
                       .find_element_by_class_name('clinic-description__text').text)

    except:
        description = 'none'


    # преимущества
    
    advantages = []    
    try:
        advantages_block = driver.find_elements_by_class_name('advantages__item') # блок
        
        for adv_i in advantages_block: # разбераю
            advantages.append(adv_i.get_attribute('textContent'))
            
    except:
        pass

    
    
    # количество врачей
    
    try:
        doc_numbers = int(driver.find_element_by_class_name('clinic-doctors')
                          .find_element_by_class_name('clinic-section__title-value')
                          .text.replace('(', '').replace(')', '')
                         )
    except:
        doc_numbers = 0




        # детальные оценки

    # оценки
    try:
        scores_block = driver.find_elements_by_class_name('rating-scale__item-line')

        scores_dict = {}
        for item in scores_block:
            temp = item.get_attribute('textContent').split(' ')
            scores_dict[temp[0]] = temp[1]


    except:
        scores_dict = {}



    # врачи


    specialist = {}


    # перехожу по ссылке
    try:
        driver.get(url + 'vrachi/#doctors')
        sleep(2)

        # считаю количество страниц
        try:
            page_number = int(driver.find_element_by_class_name('n-pagination').text.split('\n')[-1]) + 1

        except:
            page_number = 2

            # цикл для перехода    
        for i in range(1, page_number):
            driver.get(url + 'vrachi/page-' + str(i) + '/#doctors')
            sleep(2)
            doctors_block = driver.find_elements_by_class_name('clinic-doctors__card')


            for item in doctors_block:
                temp_dict = {}

                # имя
                name = item.find_element_by_class_name('object-info__title').text

                try: # средняя оценка
                    temp_dict['rating'] = item.find_element_by_class_name('rating__value').text

                except:
                    temp_dict['rating'] = 'none'


                # кол-во отзывов
                try:
                    temp_dict['review_number'] = item.find_element_by_class_name('object-info__review').text

                except:
                    temp_dict['review_number'] = 'none'

                # специализация
                try:
                     temp_dict['specialization'] = item.find_element_by_class_name('object-info__tags').text.split('\n')

                except:
                    temp_dict['specialization'] = 'none'

                # детский / взрослый
                try:
                    temp_dict['age_profile'] = specialization[-1]

                except:
                    temp_dict['age_profile'] = 'none'


                # стаж
                try:
                    temp_dict['work_experienceint'] = item.find_element_by_class_name('object-info__education').text.split('\n')

                except:
                    temp_dict['work_experienceint'] = 'none'


                # цены на прием

                doc_price = {}

                try:
                    all_price = item.find_elements_by_class_name('price-list__row') # цены на прием

                    for price in all_price:
                        serv = price.find_element_by_class_name('price-list__name').text
                        cost = price.find_element_by_class_name('price-list__value').text.replace(' ₽', '').replace(' ', '')
                        doc_price[serv] = cost

                    temp_dict['doc_price'] = doc_price

                except:
                    temp_dict['doc_price'] = doc_price


                temp_dict['doc_page'] = (item
                                         .find_element_by_class_name('object-info__title')
                                         .find_element_by_tag_name('a').get_attribute('href'))


                specialist[name] = temp_dict

    except:
        pass

    # Цены        
    full_price = {}

    # перехожу по ссылке
    try:
        driver.get(url + 'tseny/#prices')
        sleep(2)

        # считаю количество страниц
        try:
            page_number = int(driver.find_element_by_class_name('n-pagination').text.split('\n')[-1]) + 1

        except:
            page_number = 2

        for i in range(1, page_number): # цикл для перехода по страницам раздела

            driver.get(url + 'tseny/page-' + str(i) + '/#prices')
            sleep(2)

            services_block = driver.find_elements_by_class_name('services-list__info') # блоки с ценами

            for item in services_block: # разбор
                services_title = item.find_element_by_class_name('services-list__title').text # заголовок
                price = item.find_element_by_class_name('services-list__price').text.replace(' ₽', '') # цена

                full_price[services_title] = price

    except:
        pass


    data.append([url,
        verified,
        address, metro_station,
        phone, work_hours,
        clinic_type, age_profile,
        rating, review_number, scores_dict,
        main_clinic, main_clinic_page,
        advantages, description, year_f,
        doc_numbers, specialist,
        full_price,    
        clinic_table])

In [ ]:
# записываю данные в датафрейм
head_list = ['clinic_page', 'verified',
             'address', 'metro_station',
             'phone', 'work_hours',
             'clinic_type', 'age_profile',
             'rating', 'review_number', 'scores_dict',
             'main_clinic', 'main_clinic_page',
             'advantages', 'description', 'year_f',
             'doc_numbers', 'specialist',
             'full_price',
             'clinic_table']

work = pd.DataFrame(data, columns=head_list)

In [ ]:
work.shape

In [ ]:
# добавляю названия и теги

work = work.merge(works_links, how='left', on='clinic_page')

work = work[['clinic_page', 'clinic',
        'verified',
        'address', 'metro_station',
        'phone', 'work_hours',
        'clinic_type', 'age_profile', 'clinic_tags',
        'rating', 'review_number', 'scores_dict',
        'main_clinic', 'main_clinic_page',
        'advantages', 'description', 'year_f',
        'doc_numbers', 'specialist',
        'full_price',    
        'clinic_table']]

In [ ]:
work.sample(3)

In [ ]:
# формат csv, xlsx не подходит из-за ограничения кол-ва символов в ячейке

work.to_csv(file_path + '22_napopravku_data_vladivostok.csv', index=False)

In [ ]:
driver.close()